In [1]:
import pandas as pd
import numpy as np

### 类别数据

![tupian](img/categorical_methods.png)

#### 类别数据基础
类别数据的元素是不可变类型

In [3]:
fruits = ['apple', 'orange', 'apple', 'orange'] * 2
N = len(fruits)
df = pd.DataFrame({
    'fruit':fruits,
    'basket_id': np.arange(N),
    'count': np.random.randint(3, 16, size = N),
    'weight': np.random.uniform(0, 4, size = N)
})

In [5]:
df

,fruit,basket_id,count,weight
0,apple,0,9,1.253358
1,orange,1,13,2.147703
2,apple,2,7,3.861117
3,orange,3,8,1.910231
4,apple,4,8,0.907262
5,orange,5,14,3.718478
6,apple,6,4,0.797509
7,orange,7,9,0.391720


###### convert to categorical

In [14]:
fruit_cat = df['fruit'].astype('category')
print(fruit_cat)
#你也可用pd.Categorical
my_categorical = pd.Categorical(['foo', 'bar', 'baz', 'foo'])
print(my_categorical)

0     apple
1    orange
2     apple
3    orange
4     apple
5    orange
6     apple
7    orange
Name: fruit, dtype: category
Categories (2, object): [apple, orange]
[foo, bar, baz, foo]
Categories (3, object): [bar, baz, foo]


In [20]:
#categorical对象的属性
c = fruit_cat.values
print(type(c))
print(c)
print(c.categories)
print(c.codes)

<class 'pandas.core.arrays.categorical.Categorical'>
[apple, orange, apple, orange, apple, orange, apple, orange]
Categories (2, object): [apple, orange]
Index(['apple', 'orange'], dtype='object')
[0 1 0 1 0 1 0 1]


###### 从categories和codes中恢复数据

In [18]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
#用pd.Categorical.from_codes, 通过ordered参数决定是否有顺序
print(pd.Categorical.from_codes(codes, categories, ordered=True))
#你可以为没有顺序的categorical增加顺序
my_categorical.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]


[foo, bar, baz, foo]
Categories (3, object): [bar < baz < foo]

#### 用类别数据计算

In [28]:
np.random.seed(12345)
draws = np.random.randn(1000)
bins = pd.qcut(x=draws, q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
print(bins.codes[:5])
print(bins)

#use categorical with groupby
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'max', 'min'])
          .reset_index())
print(results)

print(results['quartile'])

[1 2 1 1 3]
[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]
  quartile  count       max       min
0       Q1    250 -0.685484 -2.949343
1       Q2    250 -0.010115 -0.683066
2       Q3    250  0.628894 -0.010032
3       Q4    250  3.927528  0.634238
0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]


In [34]:
#categorical的两个好处:
#1.性能提升, groupby操作用到categorical上会更快
#2.更少的内存
N = int(10e6)
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))
categories = labels.astype('category')
#查看转换前后的内存占用
print(labels.memory_usage())
print(categories.memory_usage())

80000080
10000272


In [35]:
#当然转换的过程耗时
%time _ = labels.astype('category')

CPU times: user 385 ms, sys: 145 ms, total: 530 ms
Wall time: 528 ms


#### 一些其他操作

In [37]:
s = pd.Series(list('abcd') * 2)
cats_s = s.astype('category')
print(cats_s)

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]


In [39]:
print(cats_s.cat.codes)
print(cats_s.cat.categories)

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8
Index(['a', 'b', 'c', 'd'], dtype='object')


In [40]:
#扩展categorical
actual_categories = list('abcde')
cats_s2 = cats_s.cat.set_categories(actual_categories)
print(cats_s.value_counts())
print(cats_s2.value_counts())

d    2
c    2
b    2
a    2
dtype: int64
d    2
c    2
b    2
a    2
e    0
dtype: int64


In [41]:
#缩小categorical
cats_s3 = cats_s[cats_s.isin(['a', 'b'])]
cats_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

#### to dummies

In [45]:
pd.get_dummies(cats_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


### assign and chaining methods

In [2]:
#我们用之前的例子
fruits = ['apple', 'orange', 'apple', 'orange'] * 2
N = len(fruits)
df = pd.DataFrame({
    'fruit':fruits,
    'basket_id': np.arange(N),
    'count': np.random.randint(3, 16, size = N),
    'weight': np.random.uniform(0, 4, size = N)
})

In [4]:
df

,fruit,basket_id,count,weight
0,apple,0,11,1.669587
1,orange,1,3,3.151583
2,apple,2,4,1.704342
3,orange,3,5,3.146927
4,apple,4,13,0.507562
5,orange,5,15,2.653627
6,apple,6,5,1.625252
7,orange,7,7,2.234013


#### 用assign方法来新建列

In [5]:
result = (df.assign(total_weight = df['count'] * df['weight'])
         .groupby('fruit')['total_weight']
         .mean())

#### 在使用chaining method的时候, 我们无法直接访问“中间变量”

In [7]:
#比如如下的方法无法连接起来
df_2 = df.loc[0: 5, :]
df_3 = df_2[df_2['count'] >=5]

In [9]:
#解决方案, 使用匿名函数
result = (df.loc[0:5, :]
         [lambda x: x['count']>=5])

In [10]:
result

,fruit,basket_id,count,weight
0,apple,0,11,1.669587
3,orange,3,5,3.146927
4,apple,4,13,0.507562
5,orange,5,15,2.653627


#### pipe函数
1. f(df, arg = a) 等价于 df.pipe(f, arg = a)
2. pipe函数顾名思义是可以chain的